# P-median model


Decision variables
- Whether or not to have a hospital at location $j$, $X_j$
- Whether or not to allocate customer $i$ to hospital at $j$, $Y_{ij}$

Constraints
- You can only allocate to hospitals that exist
$\forall i,j Y_{ij} \leq X_j$, alternatively $\forall j \sum_i^m Y_{ij} \leq m X_j$ 
- Each person needs to be allocated to one hospital $\forall i \sum_j^n Y_{ij} = 1$
- Need to locate exactly $p$ hospitals $\sum_j X_j = p$

Objective function
- Minimize the sum of $Y_{ij} cost(i,j)$, the sum of all the costs to reach hospital $j$ from location $i$

Note:
- The entire constraint matrix is only dependent on $p$, $n$, and $m$, the only part that actually depends on the actual lat/long locations is the objective function
- So I should write something that generates the entire constraint matrix given the constants

In [1]:
import numpy as np
from scipy.optimize import linprog

In [3]:
def p_median_constraint_matrix(num_hospitals, num_patients, num_locations):
    # n = num_locations
    # p = num_hospitals
    # m = num_patients
    # layout of decision vector
    # lentgh = n + n*m
    # first n entries -> binary, whether or not a hospital is located at location j
    # next n entries -> binary, which hospital is this patient allocated to, only one should be equal to one
    # ^ one of those sets for each patient, total = m
    vec_size = num_locations + num_locations * num_patients

    # number of constraints
    # leq constraints 
    # n constraints for the first set

    # equality constraints
    # m constraints for the second set
    # one constraint for the last set

    A_leq = np.empty((num_locations, vec_size))
    b_leq = np.empty(num_locations)

    A_eq = np.empty((num_patients + 1, vec_size))
    b_eq = np.empty(num_patients + 1)

    # first set of constraints -> all patients must be allocated to hospitals that exist
    # attempting to do the smaller constraint set to improve performance (yes i know premature optimization yada yada whatever it should be fine)
    constraint_leq_num = 0
    for j in range(num_locations):
        row = np.zeros(vec_size)
        indexes = np.array(range(vec_size))
        row[indexes % num_locations == j] = 1
        row[j] = -num_patients
        A_leq[constraint_leq_num:] = row
        b_leq[constraint_leq_num] = 0
        constraint_leq_num += 1

    # second set of constraints -> everyone is allocated to one hospital
    constraint_eq_num = 0
    for i in range(num_patients):
        row = np.zeros(vec_size)
        indexes = np.array(range(vec_size))
        row[indexes // num_locations == i + 1] = 1
        A_eq[constraint_eq_num:] = row
        b_eq[constraint_eq_num] = 1
        constraint_eq_num += 1

    # third constraint -> exactly p hospitals are allocated
    row = np.zeros(vec_size)
    indexes = np.array(range(vec_size))
    row[indexes < num_locations] = 1
    A_eq[-1:] = row
    b_eq[-1:] = num_hospitals

    bounds = np.array([(0, 1) for _ in range(vec_size)])
    integrality = np.ones(vec_size)
    return A_leq, b_leq, A_eq, b_eq, bounds, integrality

In [4]:
def p_median_objective_function(distances):
    _, num_locations = np.shape(distances)
    return np.append(np.zeros(num_locations), np.ndarray.flatten(distances))

In [38]:
def p_median_interpret_solution(solution, num_hospitals, num_patients, num_locations, output=True):
    solution_x = np.array([round(v) for v in solution.x])
    if solution.status != 0 and output:
        print(f"linprog failed: {solution.status}")
    mistakes = 0
    choices = np.array(range(num_locations))[solution_x[:num_locations] == 1]
    if output:
        print(f"Solution allocates hospitals at: {choices}")
    if len(choices) != num_hospitals:
        if output:
            print(f"!! Failed to allocate the right number of hospitals! | Allocated {len(choices)} out of {num_hospitals}")
        mistakes += 1

    vec_size = num_locations + num_locations * num_patients
    for i in range(num_patients):
        indexes = np.array(range(vec_size))
        patient_allocation = solution_x[indexes // num_locations == i + 1]
        choice = np.array(range(num_locations))[patient_allocation == 1]
        if output:
            print(f"Patient #{i} => {choice}")
        if len(choice) != 1:
            if output:
                print(f"!! Failed to properly allocate patient #{i}")
            mistakes += 1
            continue
        if not choice[0] in choices:
            if output:
                print(f"Allocated patient #{i} to a location that does not exist ({choice})")
            mistakes += 1
            continue
    return mistakes

In [14]:
A_leq, b_leq, A_eq, b_eq, bounds, integrality = p_median_constraint_matrix(num_hospitals=1, num_patients=2, num_locations=3)

# two patients, A prefers location 1, B prefers location 2 but is ok with location 1
# location 1 should get allocated
distances = np.array([[1, 5, 7], [3, 1, 7]])
c = p_median_objective_function(distances)

In [16]:
solution = linprog(c, A_leq, b_leq, A_eq, b_eq, bounds=bounds, integrality=integrality)
solution.x
p_median_interpret_solution(solution, 1, 2, 3)

Solution allocates hospitals at: [0]
Patient #0 => [0]
Patient #1 => [0]


0

In [39]:
# testing performance/reliability
mistakes = []
for _ in range(100):
    num_patients = 10
    num_locations = 10
    num_hospitals = 3
    A_leq, b_leq, A_eq, b_eq, bounds, integrality = p_median_constraint_matrix(num_hospitals=num_hospitals, num_patients=num_patients, num_locations=num_locations)
    # random distances 
    distances = np.random.rand(num_patients, num_locations)
    c = p_median_objective_function(distances)
    solution = linprog(c, A_leq, b_leq, A_eq, b_eq, bounds=bounds, integrality=integrality)
    x = p_median_interpret_solution(solution, num_hospitals=num_hospitals, num_patients=num_patients, num_locations=num_locations,output=False)
    if x > 0:
        print(distances)
        print(A_leq)
        print(b_leq)
        print(A_leq.dot(solution.x.T)) 
        print(A_eq)
        print(b_eq)
        print(A_eq.dot(solution.x.T)) 
        print("Locations")
        print(solution.x[:num_locations])
        print("Patients")
        for i in range(num_patients):
            print(solution.x[i*num_locations: (i+1)*num_locations])
        p_median_interpret_solution(solution, num_hospitals=num_hospitals, num_patients=num_patients, num_locations=num_locations,output=True)
        break
    mistakes.append(x)

In [40]:
print(f"Average # of mistakes = {sum(mistakes)/len(mistakes)}")

Average # of mistakes = 0.0


In [42]:
# testing performance/reliability
mistakes = []
for _ in range(100):
    num_patients = 20
    num_locations = 20
    num_hospitals = 3
    A_leq, b_leq, A_eq, b_eq, bounds, integrality = p_median_constraint_matrix(num_hospitals=num_hospitals, num_patients=num_patients, num_locations=num_locations)
    # random distances 
    distances = np.random.rand(num_patients, num_locations)
    c = p_median_objective_function(distances)
    solution = linprog(c, A_leq, b_leq, A_eq, b_eq, bounds=bounds, integrality=integrality)
    x = p_median_interpret_solution(solution, num_hospitals=num_hospitals, num_patients=num_patients, num_locations=num_locations, output=False);
    mistakes.append(x)

In [43]:
print(f"Average # of mistakes = {sum(mistakes)/len(mistakes)}")

Average # of mistakes = 0.0
